# 评价模型


In [9]:
import numpy as np
matrix = np.array([[1, 1/7,1/4,1/6,2], 
              [7,1,3,3,5], 
              [4,1/3,1,1/4,6], 
              [6,1/3,4,1,8],
              [1/2,1/5,1/6,1/8,1]
              ])
n=matrix.shape[0]
eigen, _= np.linalg.eig(matrix)
Ci=(np.max(np.real(eigen)))-n/(n-1)
#ri值表
ri_values={1:0.00,
           2:0.00,
           3:0.52,
           4:0.89,
           5:1.12,
           6:1.26,
           7:1.36,
           8:1.41,
           9:1.46,
           10:1.49,
           11:1.52,
           12:1.54,
           13:1.56,
           14:1.58,
           15:1.59}
Ri=ri_values[n]
CR=Ci/Ri
print("CR=",CR)

CR= 3.7496716627985083
